In [1]:
import compas_rrc as rrc
from compas.geometry import Point, Vector, Frame

# Start ROS & ABB clients
ros = rrc.RosClient()
ros.run()

abb = rrc.AbbClient(ros, '/rob1')
print('Connected to ABB IRB 6700')
    


Connected to ABB IRB 6700


In [ ]:
# done = abb.send_and_wait(rrc.SetTool('tool0'))


In [ ]:

def open_gripper():
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO3', 1))
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO4', 1))
    print(" → Gripper opened")

def close_gripper():
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO5', 0))
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO6', 0))
    print(" → Gripper closed")

# Your four key frames
frames = [
    Frame(Point(-5.28, -2174.18, 1839.08),  Vector(0.173042,0.055946,-0.983324), Vector(0.984443,-0.040716,0.170922)),  # home pos
    Frame(Point(196.362899, -1990.509986, 1560.908961),     Vector(0, 1, 0), Vector(1, 0, 0)),  # safe plane
    Frame(Point(-303.637101, -1990.509986, 1560.908961),     Vector(0,1,0), Vector(1,0,0)),  # approach pick
    Frame(Point(-303.637101, -1990.509986, 1060.908961),     Vector(0,1,0), Vector(1,0,0)),  # pick
    Frame(Point(946.362899, -1990.509986, 1560.908961),     Vector(0,1,0), Vector(1,0,0)),  # approach place
    Frame(Point(946.362899, -1990.509986, 1060.908961),     Vector(1,0,0), Vector(0,-1,0)),  # place
]

speedfly = 2000  # mm/s
speedaction = 200  # mm/s
zone  = rrc.Zone.Z20


In [8]:

# Step 1: go to Home plane (JOINT)
abb.send_and_wait(rrc.MoveToFrame(frames[0], speedfly, zone, rrc.Motion.JOINT))


'Done'

In [10]:
# Step 1: go to safe plane (JOINT)
abb.send_and_wait(rrc.MoveToFrame(frames[1], speedfly, zone, rrc.Motion.JOINT))

# Step 2: go to approach pick plane (LINEAR) > open gripper
abb.send_and_wait(rrc.MoveToFrame(frames[2], speedaction, zone, rrc.Motion.LINEAR))
open_gripper()

# Step 3: go to pick plane > close gripper (LINEAR)
abb.send_and_wait(rrc.MoveToFrame(frames[3], speedaction, zone, rrc.Motion.LINEAR))
abb.send_and_wait(rrc.WaitTime(2.0)) 
close_gripper() #add wait time here
abb.send_and_wait(rrc.WaitTime(2.0)) 

# Step 4: return to approach pick plane (LINEAR)
abb.send_and_wait(rrc.MoveToFrame(frames[2], speedaction, zone, rrc.Motion.LINEAR))


# Step 5: go to approach place plane (JOINT)
abb.send_and_wait(rrc.MoveToFrame(frames[4], speedfly, zone, rrc.Motion.JOINT))

# Step 6: go to place plane (LINEAR) > open gripper
abb.send_and_wait(rrc.MoveToFrame(frames[5], speedaction, zone, rrc.Motion.LINEAR))
abb.send_and_wait(rrc.WaitTime(2.0)) 
open_gripper()
abb.send_and_wait(rrc.WaitTime(2.0)) 

# Step 7: return to approach place plane (LINEAR)
abb.send_and_wait(rrc.MoveToFrame(frames[4], speedaction, zone, rrc.Motion.LINEAR))

# Step 8: go back to safe plane (LINEAR)
abb.send_and_wait(rrc.MoveToFrame(frames[1], speedfly, zone, rrc.Motion.LINEAR))

# Step 9: go to Home plane (JOINT)
abb.send_and_wait(rrc.MoveToFrame(frames[0], speedfly, zone, rrc.Motion.JOINT))




print("All frames completed.")


All frames completed.


In [ ]:
ros.close()
ros.terminate()
